In [116]:
import os
import dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [117]:
dotenv.load_dotenv()

True

In [118]:
language_model = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name=os.getenv("OPENAI_CHAT_MODEL_NAME"),
    temperature=float(os.getenv("OPENAI_CHAT_MODEL_TEMPERATURE")),
    max_tokens=int(os.getenv("OPENAI_CHAT_MODEL_MAX_TOKENS")),
)

In [119]:
image_prompt = "An oil painting of a group of people hiking up a mountain trail under a sky with few clouds at midday."

In [120]:
examples = [
    {
        "image_prompt": "An image of a person programming on their computer in a cozy home office with a window in the background showing a cloudy autumn afternoon landscape.",
        "title": "Autumn Coding Retreat",
    },
    {
        "image_prompt": "An image of a person hiking in an autumn forest with a backpack, wearing a jacket. The sky is cloud-filled and the sun is peaking through the clouds.",
        "title": "Autumn Trek: A Journey through the Fall Forest",
    },
]

In [121]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "artwork description: {image_prompt}"),
    ("ai", "title: '{title}'")
])

In [122]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

In [123]:
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "Please generate a short title for the artwork described by the user."),
    few_shot_prompt,
    ("user", "artwork description: {image_prompt}"),
    ("ai", "title: "),
])

print(final_prompt.format(image_prompt=image_prompt))

System: Please generate a short title for the artwork described by the user.
Human: artwork description: An image of a person programming on their computer in a cozy home office with a window in the background showing a cloudy autumn afternoon landscape.
AI: title: 'Autumn Coding Retreat'
Human: artwork description: An image of a person hiking in an autumn forest with a backpack, wearing a jacket. The sky is cloud-filled and the sun is peaking through the clouds.
AI: title: 'Autumn Trek: A Journey through the Fall Forest'
Human: artwork description: An oil painting of a group of people hiking up a mountain trail under a sky with few clouds at midday.
AI: title: 


In [124]:
chain = final_prompt | language_model | StrOutputParser()
chain.invoke({"image_prompt": image_prompt})

"'Ascend: The Midday Mountain Expedition'"